# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
!apt install libomp-dev
!pip install faiss
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (2,410 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [2]:
config = pd.read_csv('config.csv').iloc[0]

In [3]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [4]:
user = config['id']
k_value = config['k']

### Read the Data

In [5]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [ ]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [6]:
user_likes = dict()

In [7]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [8]:
users = {}
users_list = []
j = 0
for i in user_likes.keys():
  users[i] = j
  users_list.append(i)
  j+=1

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [12]:
# your code here
row = []
col = []
data = []
unique = []
for i in user_likes.keys():
  for j in user_likes[i]:
    row.append(users[i])
    col.append(j)
    data.append(1)
    unique.append(j)
unique = list(set(unique))
unique_dict = {}
for i in range(len(unique)):
  unique_dict[unique[i]] = i
for i in range(len(col)):
  col[i] = unique_dict[col[i]]
c = csr_matrix((np.array(data), (np.array(row), np.array(col))), shape=(len(user_likes.keys()),len(col)))
svd = TruncatedSVD(n_components=100, n_iter=5, random_state=10)
svd.fit(c)
a = svd.transform(c)
a = a.astype('float32')

In [13]:
len(a[users[user]])

100

In [14]:
def neighbors(user,k_value):
    ind = faiss.IndexFlatL2(100)
    ind.add(a)
    x,y = ind.search(a[users[user]:users[user]+1], k_value+1)
    return y[0][1:]

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

*I used Faiss, because the documentation was much better to understand than Scann. Faiss takes the whole matrix with all users and their likes(2D) at once. Which is actually much larger than my RAM. So, I used CSR_matrix to generate tha sparse natrix of the data we have(User Vs all unique Posts). Used TruncatedSVD to reduce the dimensionality o the matrix. Which made me to use the Faiss. Faiss is implemented by Facebook for efficient similarity search between multimedia documents. It is very efficient working with millons to even billions of datasets for similarity search ver efficiently.*

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*Fiass divides the train data to individual cells of data. When we check similarity between two data sets we check this cells to find the similarity scoree resulting in checking only these cells for query.
For Vanilla KNN, Time Complexity: O(n)
Whereas for Faiss, Time Complexity: O(n/no. of cells)*

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*Annoy is much similar to Faiss with vector dimension with 1000, i.e while using truncated svd we replace n_components with 1000. Annoy has much less precision than faiss. Whereas faiss-gpu is much powerful than faiss-cpu (Uses gpu also to compute letting it perform much better and faster computations than the cpu version).*

### Open the output file to write all the lines to the file

In [15]:
outFile = open(config['output_file'], 'w')

for n_user in neighbors(user,int(k_value)):
    user_id = users_list[n_user]
    for item_id in user_likes[user_id]:
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()